In [1]:
import pickle
import json
import os
from glob import glob
from tqdm import tqdm
from datasets import Audio
from collections import defaultdict

sr = 16000
audio = Audio(sampling_rate=sr)

In [2]:
pkls = sorted(glob('/home/husein/ssd2/processed-youtube-v2/*.pkl'))
len(pkls)

5145

In [3]:
with open('mapping-youtube-speakers-90.json') as fopen:
    unique_speakers = json.load(fopen)
    
len(unique_speakers)

10492

In [5]:
speakers = defaultdict(int)

for pkl in tqdm(pkls):
    with open(pkl, 'rb') as fopen:
        data = pickle.load(fopen)
        
    filename = os.path.split(pkl)[1].replace('.pkl', '')
        
    for result in data:
        if len(result['asr_model'][0]) < 2:
            continue
        speaker = result['classification_model'][1]
        y = audio.decode_example(audio.encode_example(result['wav_data']))['array']
        speaker_name = f'{filename}-{speaker}'
        if speaker_name not in unique_speakers:
            continue
        actual_speaker = unique_speakers[speaker_name]
        
        speakers[actual_speaker] += (len(y) / sr)

100%|███████████████████████████████████████| 5145/5145 [05:24<00:00, 15.87it/s]


In [6]:
speakers

defaultdict(int,
            {'"Abam_peluk_saya_lama_atas_pentas_akhir_MLM"-_Ali_Puteh_menangis_imbau_saat_manis_dengan_arwah_abang-_MdgGr7VD7w-speaker 0': 2.899625,
             "Main_'The_Detectives'_Dengan_Bront_Palarae_&_Fikry_Ibrahim___SEISMIK_Plays-_cYdHZBlZhM-speaker 1": 153699.4744375102,
             '"Alif_bersih_daripada_ubat_terlarang"_-_Nora_dedah_laporan_bedah_siasat_punca_kematian_anaknya-gx9JlO12W_4-speaker 1': 1.28,
             '"Anak_Melayu,_India,_Cina_hulur_RM10,_RM15"_-_Komposer_Ooi_Eow_Jin_kini_terlantar_sakit-0x5coikIeBs-speaker 0': 1.776375,
             '"Anak_Melayu,_India,_Cina_hulur_RM10,_RM15"_-_Komposer_Ooi_Eow_Jin_kini_terlantar_sakit-0x5coikIeBs-speaker 1': 1.724125,
             '"Ayah_mahu_kami_jadi_orang_berguna..._jaga_keluarga"_-_Anak_arwah_Lok_U_kenang_pesan_bapa-edwavKWfwPc-speaker 0': 2.795125,
             '"Berlakon_dengan_Opah_Aminah_paling_susah..._sumpah!"_-_Ayda_Jebat_a.k.a_Johana_Bukan_Gadis_Biasa-G8gUxijCp4o-speaker 0': 1.619625,
       

In [7]:
len({k: v for k, v in speakers.items() if v > 60})

195

In [8]:
with open('lengths-90.json', 'w') as fopen:
    json.dump(speakers, fopen)

In [1]:
import json

with open('lengths-90.json') as fopen:
    data = json.load(fopen)

In [4]:
sum(data.values()) / 60/ 60

75.22653817708583